**Sentiment Analysis of IMDB Movie Reviews**

**Problem Statement:**

In this, we have to predict the number of positive and negative reviews based on sentiments by using different classification models.

**Import necessary libraries**

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns


from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report,accuracy_score

import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from keras.models import Sequential
from keras.layers import Dense,Conv1D,MaxPooling1D
from tensorflow.keras import regularizers

from keras.models import Sequential
from keras.layers import Dense,Conv1D,MaxPooling1D
from keras.layers import LSTM,Dropout
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.callbacks import ModelCheckpoint


import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import LancasterStemmer,WordNetLemmatizer

from bs4 import BeautifulSoup
import re,string,unicodedata



import os


# fix random seed for reproducibility
np.random.seed(7)

In [ ]:
#Load the libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import random
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
#from wordcloud import WordCloud,STOPWORDS
#from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
#import spacy
import re,string,unicodedata
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import LancasterStemmer,WordNetLemmatizer
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
#from textblob import TextBlob
#from textblob import Word
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
import numpy as np
from sklearn.model_selection import train_test_split
import nltk
nltk.download('stopwords')


import os



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Import the training dataset**

In [ ]:
# we now load imdb_data.csv with pandas librairies which will help us to beter manipulate data in csv format and matrix

imdb_data = pd.read_csv('/content/drive/MyDrive/dataset/IMDB Dataset.csv')

In [ ]:
imdb_data.head(10)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
5,"Probably my all-time favorite movie, a story o...",positive
6,I sure would like to see a resurrection of a u...,positive
7,"This show was an amazing, fresh & innovative i...",negative
8,Encouraged by the positive comments about this...,negative
9,If you like original gut wrenching laughter yo...,positive


**Exploratery data analysis**

In [ ]:
#Summary of the dataset
imdb_data.describe()

,review,sentiment
count,50000,50000
unique,49582,2
top,Loved today's show!!! It was a variety and not...,negative
freq,5,25000


**Sentiment count**

In [ ]:
#sentiment count

imdb_data['sentiment'].value_counts()

negative    25000
positive    25000
Name: sentiment, dtype: int64

We can see that the dataset is balanced.

**Data preprocessing**

Text preprocessing is traditionally an important step for natural language processing (NLP) tasks. It transforms text into a more digestible form so that machine learning algorithms can perform better. They are many ways to do preprocessing task, as example, TextVectorization layer uses in keras; So in our case, we will do it in 03 step: standardization, tokenization and vectorisation.

**Text Normalization**


Normalization refers to preprocessing text, typically to remove punctuation or HTML elements to simplify the dataset.As you saw above, the reviews contain various HTML tags such as <br / ,so in this step, we will clean reviews.

**Removing html strips and noise text**

In [ ]:
#Removing the html strips
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

#Removing the square brackets
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

#Removing the noisy text
def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    return text
#Apply function on review column
imdb_data['review']=imdb_data['review'].apply(denoise_text)

**Removing special characters**

In [ ]:
#Define function for removing special characters
def remove_special_characters(text, remove_digits=True):
    pattern=r'[^a-zA-z0-9\s]'
    text=re.sub(pattern,'',text)
    return text
#Apply function on review column
imdb_data['review']=imdb_data['review'].apply(remove_special_characters)

In [ ]:
imdb_data.head(5)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production The filming tech...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically theres a family where a little boy J...,negative
4,Petter Matteis Love in the Time of Money is a ...,positive


**Text stemming**

In [ ]:
#Stemming the text
def simple_stemmer(text):
    ps=nltk.porter.PorterStemmer()
    text= ' '.join([ps.stem(word) for word in text.split()])
    return text
#Apply function on review column
imdb_data['review']=imdb_data['review'].apply(simple_stemmer)

In [ ]:
imdb_data.head(10)

,review,sentiment
0,one of the other review ha mention that after ...,positive
1,A wonder littl product the film techniqu is ve...,positive
2,I thought thi wa a wonder way to spend time on...,positive
3,basic there a famili where a littl boy jake th...,negative
4,petter mattei love in the time of money is a v...,positive
5,probabl my alltim favorit movi a stori of self...,positive
6,I sure would like to see a resurrect of a up d...,positive
7,thi show wa an amaz fresh innov idea in the 70...,negative
8,encourag by the posit comment about thi film o...,negative
9,If you like origin gut wrench laughter you wil...,positive


**Removing stopwords**

In [ ]:
#set stopwords to english
stop=set(stopwords.words('english'))
print(stop)

#removing the stopwords
def remove_stopwords(text, is_lower_case=False):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text
#Apply function on review column
imdb_data['review']=imdb_data['review'].apply(remove_stopwords)

{"won't", "doesn't", 'over', 'by', 'some', 'ma', 'each', 'shan', "hasn't", 'he', "it's", 'mightn', 'are', 'before', "you'd", 'its', 'those', 'do', 'any', 'should', 's', 'below', "mustn't", 'll', 'few', 'you', 'after', 'be', 'my', 'for', 'yours', "you're", 'so', 'what', "weren't", 'was', 'ours', 'hasn', 've', 't', "aren't", 'with', 'from', 'too', "couldn't", 'me', "you've", 'a', 'himself', 'yourself', 'hers', 'that', 'own', 'above', 'yourselves', "she's", 'but', 'or', 'very', 'why', 'isn', 'same', "wasn't", 'theirs', 'such', 'him', 'm', 'how', 'all', 'more', 'this', 'don', "shan't", 're', 'to', 'wouldn', 'o', 'am', 'mustn', 'we', 'haven', 'didn', "shouldn't", 'on', 'in', 'needn', 'they', 'of', "wouldn't", "you'll", "mightn't", 'can', 'further', 'them', 'whom', 'under', 'both', 'where', 'i', 'themselves', 'being', 'herself', 'has', 'itself', 'couldn', 'most', 'doing', 'up', 'during', 'won', 'his', 'nor', 'hadn', 'shouldn', 'no', 'y', 'the', 'there', "needn't", 'were', "that'll", 'about',

Texte normalization

In [ ]:
#Tokenization of text
tokenizer=ToktokTokenizer()
#Setting English stopwords
stopword_list=nltk.corpus.stopwords.words('english')

After all this step, we obtain the following clean dataset

In [ ]:
imdb_data.head(10)

**Splitting data**

Let's split data into train an test set

In [ ]:
X=imdb_data['review'].to_numpy()

In [ ]:
y=imdb_data['sentiment'].to_numpy()

In [ ]:
y = np.array(list(map(lambda x: 1 if x=="positive" else 0, y)))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

**Tokenization**

Tokenization refers to the splitting of strings into tokens (for example, dividing a sentence into individual words, splitting it into spaces).

In [ ]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words = 5000) # it will also help to made tokenization
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [ ]:
X_train

[[1,
  4,
  8,
  513,
  481,
  653,
  119,
  1803,
  178,
  212,
  103,
  15,
  72,
  20,
  858,
  68,
  4284,
  6,
  1822,
  3698,
  249,
  447,
  867,
  55,
  55,
  1449,
  51,
  12,
  756,
  89,
  27,
  316,
  320,
  1186,
  1240,
  11,
  206,
  313,
  55,
  365,
  56,
  1695,
  38,
  898,
  3735,
  487,
  151,
  525,
  807,
  2,
  296,
  4173,
  343,
  353,
  1639,
  1096,
  223,
  1157,
  275,
  1014,
  1036,
  86,
  786,
  354,
  734,
  2322,
  281,
  433,
  747,
  390,
  303,
  551,
  298,
  960,
  1096,
  923,
  270,
  35,
  298,
  58,
  319,
  88,
  298,
  58,
  3698,
  3931,
  4669,
  328],
 [155,
  123,
  899,
  697,
  917,
  25,
  159,
  312,
  994,
  3204,
  226,
  232,
  123,
  25,
  539,
  129,
  364,
  2848,
  1866,
  187,
  4001,
  345,
  658,
  1682,
  1274,
  906,
  1380,
  1122,
  2519,
  487,
  151,
  1578,
  3614,
  1477,
  3481,
  351,
  263],
 [734,
  1210,
  696,
  2,
  86,
  2712,
  20,
  508,
  97,
  119,
  2807,
  2727,
  1133,
  144,
  1847,
  1392,
  341,


Since the sentences are of different length, we pad our reviews to make them have same lengths 

In [ ]:
top_words = 10000

# truncate and pad input sequences
max_review_length = 500
X_train = sequence.pad_sequences(X_train, padding = 'post',maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, padding = 'post',maxlen=max_review_length)


Let's build models

Sequence to sequence learning has been successful in many tasks such as machine translation, speech recognition, sentiments analysis etc. The dominant approach to date encodes the input sequence with a series of bi-directional recurrent neural networks (RNN) and generates a variable length output with another set of decoder RNNs, both of which interface via a soft-attention mechanism.

No more chatter, in this part, we will see how work rnn models and cnn model in how binary text classification

**Simple Recurrent Neural Network**

In [ ]:
max_features = 10000
embedding_dim = 32

model = tf.keras.Sequential([
  layers.Embedding(max_features + 1, embedding_dim),
  layers.Dropout(0.2),
  layers.GlobalAveragePooling1D(),
  layers.Dropout(0.2),
  layers.Dense(1)])

model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, None, 32)          320032    
_________________________________________________________________
dropout_8 (Dropout)          (None, None, 32)          0         
_________________________________________________________________
global_average_pooling1d_4 ( (None, 32)                0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 33        
Total params: 320,065
Trainable params: 320,065
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(loss=losses.BinaryCrossentropy(from_logits=True),
              optimizer='adam',
              metrics=tf.metrics.BinaryAccuracy(threshold=0.0))

In [ ]:
# Train the model
epochs = 10
history = model.fit(
    X_train, y_train,
    validation_split = 0.2,
    epochs=epochs)

Epoch 1/10
1000/1000 [==============================] - 13s 12ms/step - loss: 0.6770 - binary_accuracy: 0.6244 - val_loss: 0.5649 - val_binary_accuracy: 0.8129
Epoch 2/10
1000/1000 [==============================] - 12s 12ms/step - loss: 0.5210 - binary_accuracy: 0.8268 - val_loss: 0.4294 - val_binary_accuracy: 0.8468
Epoch 3/10
1000/1000 [==============================] - 13s 13ms/step - loss: 0.3996 - binary_accuracy: 0.8634 - val_loss: 0.3621 - val_binary_accuracy: 0.8651
Epoch 4/10
1000/1000 [==============================] - 13s 13ms/step - loss: 0.3398 - binary_accuracy: 0.8808 - val_loss: 0.3275 - val_binary_accuracy: 0.8769
Epoch 5/10
1000/1000 [==============================] - 12s 12ms/step - loss: 0.2992 - binary_accuracy: 0.8910 - val_loss: 0.3091 - val_binary_accuracy: 0.8795
Epoch 6/10
1000/1000 [==============================] - 12s 12ms/step - loss: 0.2787 - binary_accuracy: 0.8958 - val_loss: 0.2976 - val_binary_accuracy: 0.8817
Epoch 7/10
1000/1000 [==================

In [ ]:
# Final evaluation of the model

scores = model.evaluate(X_test, y_test, verbose=1,batch_size = 256)
print("Accuracy: %.2f%%" % (scores[1]*100))

40/40 [==============================] - 0s 8ms/step - loss: 0.2902 - binary_accuracy: 0.8845
Accuracy: 88.45%


**CNN**

 CNNs are basically just several layers of convolutions with nonlinear activation functions like ReLU or tanh applied to the results. In a traditional feedforward neural network we connect each input neuron to each output neuron in the next layer(fully connected layer). In CNNs we don’t do that; Instead, we use convolutions over the input layer to compute the output. This results in local connections, where each region of the input is connected to a neuron in the output.

In [ ]:
from keras.layers import GlobalMaxPooling1D

In [ ]:
model = Sequential()

embedding_layer = Embedding(max_features + 1, 100, input_length=max_review_length , trainable=False)
model.add(embedding_layer)

model.add(Conv1D(128, 5, activation='relu'))

model.add(GlobalMaxPooling1D())
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [ ]:
history = model.fit(X_train, y_train, batch_size=128, epochs=10, verbose=1, validation_split=0.2)

Epoch 1/10
250/250 [==============================] - 95s 378ms/step - loss: 0.6738 - acc: 0.6062 - val_loss: 0.5610 - val_acc: 0.7315
Epoch 2/10
250/250 [==============================] - 94s 378ms/step - loss: 0.5221 - acc: 0.7572 - val_loss: 0.4887 - val_acc: 0.7696
Epoch 3/10
250/250 [==============================] - 94s 378ms/step - loss: 0.4505 - acc: 0.7975 - val_loss: 0.4459 - val_acc: 0.7922
Epoch 4/10
250/250 [==============================] - 94s 377ms/step - loss: 0.3980 - acc: 0.8270 - val_loss: 0.4216 - val_acc: 0.8051
Epoch 5/10
250/250 [==============================] - 94s 377ms/step - loss: 0.3578 - acc: 0.8529 - val_loss: 0.4062 - val_acc: 0.8117
Epoch 6/10
250/250 [==============================] - 94s 376ms/step - loss: 0.3319 - acc: 0.8673 - val_loss: 0.3938 - val_acc: 0.8192
Epoch 7/10
250/250 [==============================] - 94s 376ms/step - loss: 0.3075 - acc: 0.8807 - val_loss: 0.3840 - val_acc: 0.8229
Epoch 8/10
250/250 [==============================] - 9

In [ ]:
score = model.evaluate(X_test, y_test, verbose=1)

313/313 [==============================] - 11s 36ms/step - loss: 0.3576 - acc: 0.8408


**CNN with one LSTM layer**

We can easily add a one-dimensional CNN and max pooling layers after the Embedding layer which then feed the consolidated features to the LSTM. We can use a smallish set of 32 features with a small filter length of 5.

In [ ]:
# create the model
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(Conv1D(128, kernel_size=5, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(128, dropout=0.5, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())


Model: "sequential_27"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_25 (Embedding)     (None, 500, 32)           320000    
_________________________________________________________________
conv1d_13 (Conv1D)           (None, 500, 128)          20608     
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 250, 128)          0         
_________________________________________________________________
lstm_15 (LSTM)               (None, 128)               131584    
_________________________________________________________________
dense_19 (Dense)             (None, 1)                 129       
Total params: 472,321
Trainable params: 472,321
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
history = model.fit(X_train, y_train, epochs=5, batch_size=128,verbose = 1,validation_split = 0.2)

Epoch 1/5
250/250 [==============================] - 387s 2s/step - loss: 0.6934 - accuracy: 0.5044 - val_loss: 0.6932 - val_accuracy: 0.5013
Epoch 2/5
250/250 [==============================] - 386s 2s/step - loss: 0.6932 - accuracy: 0.5027 - val_loss: 0.6931 - val_accuracy: 0.5014
Epoch 3/5
250/250 [==============================] - 381s 2s/step - loss: 0.6929 - accuracy: 0.5061 - val_loss: 0.6935 - val_accuracy: 0.5014
Epoch 4/5
250/250 [==============================] - 378s 2s/step - loss: 0.6929 - accuracy: 0.4961 - val_loss: 0.6935 - val_accuracy: 0.4985
Epoch 5/5
250/250 [==============================] - 386s 2s/step - loss: 0.6935 - accuracy: 0.5053 - val_loss: 0.6934 - val_accuracy: 0.5014


In [ ]:
# Final evaluation of the model

scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 50.48%
